In [35]:
from desc.transform import Transform
from desc.equilibrium import Equilibrium
from desc.basis import ChebyshevZernikeBasis, chebyshev_z
from desc.compute import compute
from desc.grid import LinearGrid, ConcentricGrid, QuadratureGrid
from desc.compute.utils import get_transforms

In [12]:
eq = Equilibrium(L=3,M=3,N=3,mirror=True)

In [28]:
grid = LinearGrid(L=3,M=3,N=3)

In [33]:
tranf = Transform(basis=eq.R_basis,grid=grid,method="jitable")
# jitable is direct1

In [40]:
get_transforms("B",eq,grid,jitable=True)

{'grid': LinearGrid at 0x291c07d50 (L=3, M=3, N=3, NFP=1, sym=False, node_pattern=linear),
 'R': Transform at 0x293124350 (method=jitable, basis=ChebyshevZernikeBasis at 0x17f9a5890 (L=3, M=3, N=3, NFP=1, sym=False, spectral_indexing=ansi), grid=LinearGrid at 0x291c07d50 (L=3, M=3, N=3, NFP=1, sym=False, node_pattern=linear)),
 'Z': Transform at 0x293195c90 (method=jitable, basis=ChebyshevZernikeBasis at 0x17f7c8590 (L=3, M=3, N=3, NFP=1, sym=False, spectral_indexing=ansi), grid=LinearGrid at 0x291c07d50 (L=3, M=3, N=3, NFP=1, sym=False, node_pattern=linear)),
 'L': Transform at 0x29315acd0 (method=jitable, basis=ChebyshevZernikeBasis at 0x174a4da90 (L=3, M=3, N=3, NFP=1, sym=False, spectral_indexing=ansi), grid=LinearGrid at 0x291c07d50 (L=3, M=3, N=3, NFP=1, sym=False, node_pattern=linear))}

In [42]:
eq.compute("B")["B"]

Array([[0.        , 0.31830989, 0.        ],
       [0.        , 0.31830989, 0.        ],
       [0.        , 0.31830989, 0.        ],
       ...,
       [0.        , 0.31830989, 0.        ],
       [0.        , 0.31830989, 0.        ],
       [0.        , 0.31830989, 0.        ]], dtype=float64)